In [15]:
import pandas as pd
from scipy.stats import norm
import numpy as np
from scipy.optimize import minimize
from numpy import log, exp, sqrt
pd.set_option('display.float_format', '{:.4f}'.format)

data = pd.read_excel('../data/Options_call_put.xlsx', sheet_name='Sheet1')

'''
- Risk free rate: 7.2 
- Calendar days before execution: 70 
- Spot price: 90.2 
- Futures price for stock 91.1 
- Futures price for currencies 89.3
'''

for col in data.columns:
    if col != 'Strike':
        data[col] = data[col].apply(lambda x: x / 1000)

_r = 0.072
_r_f = 0.1
_T = 70/365
_S = 90.2
_FC = 89.3
data['Call: mid'] = (data['Call: bid'] + data['Call: offer'])/2
data['Put: mid'] = (data['Put: bid'] + data['Put: offer'])/2


In [16]:
data = data[50:70]

In [17]:
class BsmPricing:
    '''
    Class for pricing EUROPEAN options using the Black-Scholes-Merton model.

    _calculate_d1_d2: Internal method to compute the d1 and d2 coefficients.

    price: Computes the European option price.

    delta: Computes the option's sensitivity to underlying asset price changes.

    gamma: Computes the rate of change in delta with respect to changes in the underlying price.

    vega: Computes the option's sensitivity to volatility changes.

    volga: Computes the rate of change in vega with respect to changes in volatility.

    vanna: Computes the rate of change in vega with respect to changes in the underlying price.
    
    theta: Computes the rate of change in option price with respect to time decay.
    '''
    def __init__(self, S0, K, T, r, b, sigma, is_call=True):
        self.S0 = S0
        self.K = K
        self.T = T
        self.b = b
        self.r = r
        self.sigma = sigma
        self.is_call = is_call
        self._calculate_d1_d2()

    def _calculate_d1_d2(self):
        self.d1 = (log(self.S0 / self.K) + (self.b + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * sqrt(self.T))
        self.d2 = self.d1 - self.sigma * sqrt(self.T)

    def price(self):
        if self.is_call:
            price = (self.S0 * exp((self.b - self.r) * self.T) * norm.cdf(self.d1) -
                    self.K * exp(-self.r * self.T) * norm.cdf(self.d2))
        else:
            price = (self.K * exp(-self.r * self.T) * norm.cdf(-self.d2) -
                    self.S0 * exp((self.b - self.r) * self.T) * norm.cdf(-self.d1))
        return price

    def delta(self):
        if self.is_call:
            return exp((self.b - self.r) * self.T) * norm.cdf(self.d1)
        else:
            return -exp((self.b - self.r) * self.T) * norm.cdf(-self.d1)

    def gamma(self):
        return norm.pdf(self.d1) * exp((self.b - self.r) * self.T) / (self.S0 * self.sigma * sqrt(self.T))

    def vega(self):
        return self.S0 * exp((self.b - self.r) * self.T) * norm.pdf(self.d1) * sqrt(self.T)

    def volga(self):
        return self.S0 * exp((self.b - self.r) * self.T) * norm.pdf(self.d1) * sqrt(self.T) * self.d1 * self.d2 / self.sigma

    def vanna(self):
        return -exp((self.b - self.r) * self.T) * norm.pdf(self.d1) * self.d2 / self.sigma

    def theta(self):
        if self.is_call:
            theta = (-exp((self.b - self.r) * self.T) * self.S0 * norm.pdf(self.d1) * self.sigma / (2 * sqrt(self.T)) 
                     - (self.b - self.r) * self.S0 * exp((self.b - self.r) * self.T) * norm.cdf(self.d1) 
                     - self.r * self.K * exp(-self.r * self.T) * norm.cdf(self.d2))
        else:
            theta = (-exp((self.b - self.r) * self.T) * self.S0 * norm.pdf(self.d1) * self.sigma / (2 * sqrt(self.T)) 
                     + (self.b - self.r) * self.S0 * exp((self.b - self.r) * self.T) * norm.cdf(-self.d1) 
                     + self.r * self.K * exp(-self.r * self.T) * norm.cdf(-self.d2))
        return theta


In [18]:
class BsPricing:
    '''
    Class for pricing AMERICAN options using the Bjerksund Stensland (1993) model.

    price: Computes the American option price using the Bjerksund Stensland (1993) model.

    _bsm_price: Computes the European option price using the Black-Scholes-Merton model.

    delta: Computes the option's sensitivity to underlying asset price changes.

    gamma: Computes the rate of change in delta with respect to changes in the underlying price.

    vega: Computes the option's sensitivity to volatility changes.
    
    theta: Computes the rate of change in option price with respect to time decay.

    vanna: Computes the rate of change in delta with respect to changes in volatility.

    volga: Computes the rate of change in vega with respect to changes in volatility.
    
    _numerical_derivative: Utility method to compute the numerical derivative of a given function with respect to a specified variable.
    '''
    def __init__(self, S0, K, T, r, b, sigma, is_call=True):
        self.S0 = S0
        self.K = K
        self.T = T
        self.b = b
        self.r = r
        self.sigma = sigma
        self.is_call = is_call

    def price(self):
        if not self.is_call:
            return BsPricing(self.K, self.S0, self.T, self.r-self.b, -self.b, self.sigma, is_call=True).price()
        
        if self.b >= self.r:
            return self._bsm_price()
        
        sigma2 = self.sigma * self.sigma

        def phi(s, t, gamma, h, i):
            lambd = (-self.r + gamma * self.b + 0.5 * gamma * (gamma - 1) * sigma2) * t
            d = - (log(s/h) + (self.b + (gamma - 0.5) * sigma2) * t) / (self.sigma * sqrt(t))
            k = 2 * self.b / sigma2 + (2 * gamma - 1)
            return exp(lambd) * s ** gamma * (norm.cdf(d) - ((i/s) ** k) * norm.cdf(d - 2 * log(i/s) / (self.sigma * sqrt(t))))
        
        beta = 0.5 - self.b / sigma2 + sqrt((self.b / sigma2 - 0.5) ** 2 + 2 * self.r / sigma2)
        B_0 = max(self.K, self.r / (self.r - self.b) * self.K)
        B_inf = beta / (beta - 1) * self.K

        def H(t):
            return -(self.b * t + 2 * self.sigma * sqrt(t)) * B_0 / (B_inf - B_0) 
        
        I = B_0 + (B_inf - B_0) * (1 - exp(H(self.T)))
        alpha = (I - self.K) * I ** (-beta)

        return alpha * self.S0 ** beta - alpha * phi(self.S0, self.T, beta, I, I) + phi(self.S0, self.T, 1, I, I) - phi(self.S0, self.T, 1, self.K, I) - self.K * phi(self.S0, self.T, 0, I, I) + self.K * phi(self.S0, self.T, 0, self.K, I)

    def _bsm_price(self):
        d1 = (log(self.S0 / self.K) + (self.b + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * sqrt(self.T))
        d2 = d1 - self.sigma * sqrt(self.T)
        if self.is_call:
            price = (self.S0 * exp((self.b - self.r) * self.T) * norm.cdf(d1) -
                    self.K * exp(-self.r * self.T) * norm.cdf(d2))
        else:
            price = (self.K * exp(-self.r * self.T) * norm.cdf(-d2) -
                    self.S0 * exp((self.b - self.r) * self.T) * norm.cdf(-d1))
        return price

    def delta(self):
        return self._numerical_derivative('price', 'S0', eps=1e-2 * self.S0)
    
    def gamma(self):
        return self._numerical_derivative('delta', 'S0', eps=1e-2 * self.S0)

    def vega(self):
        return self._numerical_derivative('price', 'sigma', eps=1e-2)

    def theta(self):
        return -self._numerical_derivative('price', 'T', eps=1e-2)

    def vanna(self):
        return self._numerical_derivative('delta', 'sigma', eps=1e-2)

    def volga(self):
        return self._numerical_derivative('vega', 'sigma', eps=1e-2)

    def _numerical_derivative(self, func, var, eps=1e-2):
        params = {'S0': self.S0, 'K': self.K, 'T': self.T, 'r': self.r, 
                  'b': self.b, 'sigma': self.sigma, 
                  'is_call': self.is_call}
        params[var] += eps
        obj1 = BsPricing(**params)
        f = getattr(obj1, func)
        f_plus_eps = f()
        params[var] -= 2 * eps
        obj2 = BsPricing(**params)
        f = getattr(obj2, func)
        f_minus_eps = f()
        return (f_plus_eps - f_minus_eps) / (2 * eps)
    


In [19]:
class MonteCarloPricing:
    '''
    Class for pricing EUROPEAN options using the Monte Carlo simulation method.

    price: Computes the European option price using Monte Carlo simulation.

    delta: Computes the option's sensitivity to underlying asset price changes.

    gamma: Computes the rate of change in delta with respect to changes in the underlying price.

    vega: Computes the option's sensitivity to volatility changes.

    theta: Computes the rate of change in option price with respect to time decay.

    vanna: Computes the rate of change in delta with respect to changes in volatility.

    volga: Computes the rate of change in vega with respect to changes in volatility.
    
    _numerical_derivative: Utility method to compute the numerical derivative of a given function with respect to a specified variable.
    '''
    def __init__(self, S0, K, T, r, b, sigma, is_call=True, n=1000000):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.b = b
        self.sigma = sigma
        self.is_call = is_call
        self.n = n

    def price(self):
        z = np.random.standard_normal(self.n)
        ST = self.S0 * exp((self.b - 0.5 * self.sigma**2) * self.T + self.sigma * sqrt(self.T) * z)
        
        if self.is_call:
            payoff = np.maximum(ST - self.K, 0)
        else:
            payoff = np.maximum(self.K - ST, 0)
        
        return exp(-self.r * self.T) * np.mean(payoff)

    def delta(self):
        return self._numerical_derivative(self.price, 'S0', eps=1e-2 * self.S0)
    
    def gamma(self):
        return self._numerical_derivative(self.delta, 'S0', eps=1e-2 * self.S0)

    def vega(self):
        return self._numerical_derivative(self.price, 'sigma', eps=1e-2)

    def theta(self):
        return -self._numerical_derivative(self.price, 'T', eps=1e-2)
    
    def vanna(self):
        return self._numerical_derivative(self.delta, 'sigma', eps=1e-2)
    
    def volga(self):
        return self._numerical_derivative(self.vega, 'sigma', eps=1e-2)

    def _numerical_derivative(self, func, var, eps=1e-2):
        setattr(self, var, getattr(self, var) + eps)
        f_plus_eps = func()
        setattr(self, var, getattr(self, var) - 2 * eps)
        f_minus_eps = func()
        setattr(self, var, getattr(self, var) + eps)
        return (f_plus_eps - f_minus_eps) / (2 * eps)


In [20]:
class TrinomialTreePricing:
    '''
    Class for pricing options using the Trinomial Tree method, suitable for both EUROPEAN and AMERICAN options.

    _traverse_tree: Internal method to traverse the trinomial tree and compute option price.

    price: Computes the option price using the trinomial tree method.

    delta: Computes the option's sensitivity to underlying asset price changes.

    gamma: Computes the rate of change in delta with respect to changes in the underlying price.

    vega: Computes the option's sensitivity to volatility changes.

    theta: Computes the rate of change in option price with respect to time decay.

    vanna: Computes the rate of change in delta with respect to changes in volatility.

    volga: Computes the rate of change in vega with respect to changes in volatility.
    
    _numerical_derivative: Utility method to compute the numerical derivative of a given function with respect to a specified variable.
    '''
    def __init__(self, S0, K, T, r, b, sigma, is_call=True, is_european=True, n=100):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.b = b
        self.sigma = sigma
        self.n = n
        self.dt = T/n
        self.discount = exp(-r * self.dt)
        self.is_call = is_call
        self.is_european = is_european

        self.u = exp(sigma * sqrt(2 * self.dt))
        self.pu = ((exp(b * self.dt/2) - exp(-sigma * sqrt(self.dt/2))) / 
                   (exp(sigma * sqrt(self.dt/2)) - exp(-sigma * sqrt(self.dt/2))))**2
        self.pd = ((exp(sigma * sqrt(self.dt/2)) - exp(b * self.dt/2)) / 
                   (exp(sigma * sqrt(self.dt/2)) - exp(-sigma * sqrt(self.dt/2))))**2
        self.pm = 1 - self.pu - self.pd

        self.tree = np.zeros((n+1, 2*n+1))

        for i in range(self.n + 1):
            for j in range(2 * i + 1):
                self.tree[i, j] = self.S0 * (self.u**(j - i))

        self.payoffs = np.maximum(0, self.tree[-1] - self.K) if self.is_call else np.maximum(0, self.K - self.tree[-1])
    
    def _traverse_tree(self):
        payoffs = self.payoffs.copy()
        for i in range(self.n-1, -1, -1):
            new_payoffs = np.zeros(2*i + 1)
            for j in range(2*i + 1):
                if self.is_european:
                    new_payoffs[j] = (self.pd * payoffs[j] + 
                                    self.pm * payoffs[j+1] + 
                                    self.pu * payoffs[j+2]) * self.discount
                else:
                    if self.is_call:
                        exercise_price = max(0, self.tree[i, j] - self.K)
                    else:
                        exercise_price = max(0, self.K - self.tree[i, j])
                    new_payoffs[j] = max(exercise_price, 
                                        (self.pd * payoffs[j] + 
                                        self.pm * payoffs[j+1] + 
                                        self.pu * payoffs[j+2]) * self.discount)
            payoffs = new_payoffs
        return payoffs[0]
    
    def price(self):
        return self._traverse_tree()
    
    def delta(self):
        return self._numerical_derivative('price', 'S0', eps=1e-2 * self.S0)
    
    def gamma(self):
        return self._numerical_derivative('delta', 'S0', eps=1e-2 * self.S0)

    def vega(self):
        return self._numerical_derivative('price', 'sigma', eps=1e-2)

    def theta(self):
        return -self._numerical_derivative('price', 'T', eps=1e-2)

    def vanna(self):
        return self._numerical_derivative('delta', 'sigma', eps=1e-2)

    def volga(self):
        return self._numerical_derivative('vega', 'sigma', eps=1e-2)

    def _numerical_derivative(self, func, var, eps=1e-2):
        params = {'S0': self.S0, 'K': self.K, 'T': self.T, 'r': self.r, 
                  'b': self.b, 'sigma': self.sigma, 'n': self.n, 
                  'is_call': self.is_call, 'is_european': self.is_european}
        params[var] += eps
        obj1 = TrinomialTreePricing(**params)
        f = getattr(obj1, func)
        f_plus_eps = f()
        params[var] -= 2 * eps
        obj2 = TrinomialTreePricing(**params)
        f = getattr(obj2, func)
        f_minus_eps = f()
        return (f_plus_eps - f_minus_eps) / (2 * eps)

In [21]:
'''
Function to compute option price and Greeks for a given pricing model.
compute_price_and_greeks:

- pricing_type: Model type ('BS' for Black-Scholes, 'MC' for Monte Carlo, 'TT' for Trinomial Tree).

- Returns a pandas Series with option value and Greeks.


Function to compute implied volatility for a given market price using a specified pricing model.
implied_volatility:

- pricing_type: Model type ('BS' for Black-Scholes, 'TT' for Trinomial Tree, 'MC' for Monte Carlo).

- market_price: Observed market price of the option.

- Returns the implied volatility.


Function to process data and compute implied volatilities and Greeks for call and put options.
process_data:

- pricing_type: Model type ('BS' for Black-Scholes, 'MC' for Monte Carlo, 'TT' for Trinomial Tree).

- is_european: Boolean indicating if the option is European (default is True).

- is_stock: Boolean indicating if the underlying is a stock (default is True).

- Returns two dataframes, one for call options and one for put options, with implied volatilities and Greeks.
'''


def compute_price_and_greeks(pricing_type, is_call, is_european, S, K, T, r, b, sigma):
        if pricing_type == 'BS':
            option = (BsmPricing(S, K, T, r, b, sigma, is_call) if is_european 
                      else BsPricing(S, K, T, r, b, sigma, is_call) )
        elif pricing_type == 'MC':
            assert is_european, 'Monte Carlo simulation only supports European options'
            option = (MonteCarloPricing(S, K, T, r, b, sigma, is_call))
        elif pricing_type == 'TT':
            option = (TrinomialTreePricing(S, K, T, r, b, sigma, is_call, is_european))
        
        if sigma != 0:
            return pd.Series({
                'Value': option.price(),
                'Delta': option.delta(),
                'Gamma': option.gamma(),
                'Vega': option.vega(),
                'Volga': option.volga(),
                'Vanna': option.vanna(),
                'Theta': option.theta()
            })
        else:
            return pd.Series({
                'Value': np.nan,
                'Delta': np.nan,
                'Gamma': np.nan,
                'Vega': np.nan,
                'Volga': np.nan,
                'Vanna': np.nan,
                'Theta': np.nan
            })
             

def implied_volatility(pricing_type, market_price, S, K, T, r, b, is_call, is_european=True):
    if pricing_type == 'BS':
        def price(S, K, T, r, b, sigma, is_call):
            if is_european:
                option = BsmPricing(S, K, T, r, b, sigma, is_call)
                return option.price()
            else:
                option = BsPricing(S, K, T, r, b, sigma, is_call)
                return option.price()
                    
    elif pricing_type == 'TT' or pricing_type == 'MC':
        def price(S, K, T, r, b, sigma, is_call):
            option = TrinomialTreePricing(S, K, T, r, b, sigma, is_call, is_european)
            return option.price()

    error_function = lambda sigma: (price(S, K, T, r, b, sigma, is_call) - market_price)**2
    result = minimize(error_function, 0.2, bounds=[(0.1, 10)])
    return result.x[0]

def process_data(pricing_type, is_european=True, is_stock=True):
    if is_stock:
        b = _r
        S = _S
    else:
        b = _r - _r_f
        S = _FC

    data_call = data[['Strike','Call: bid', 'Call: offer', 'Call: mid']]
    data_put = data[['Strike','Put: bid', 'Put: offer', 'Put: mid']]

    data_call = data_call.copy()
    data_call['IV'] = data_call.apply(lambda row: implied_volatility(pricing_type, row['Call: mid'], S, row['Strike'], _T, _r, b, True)
                                                            if not np.isnan(row['Call: mid']) else np.nan, axis=1)
    data_call.dropna(inplace=True)
    data_call.reset_index(drop=True, inplace=True)

    data_put = data_put.copy()
    data_put['IV'] = data_put.apply(lambda row: implied_volatility(pricing_type, row['Put: mid'], S, row['Strike'], _T, _r, b, False)
                                                            if not np.isnan(row['Put: mid']) else np.nan, axis=1)
    data_put.dropna(inplace=True)
    data_put.reset_index(drop=True, inplace=True)

    greeks_call_df = data_call.apply(lambda row: compute_price_and_greeks(pricing_type, True, is_european, S, row['Strike'], _T, _r, b, row['IV']), axis=1)
    greeks_put_df = data_put.apply(lambda row:  compute_price_and_greeks(pricing_type, False, is_european, S, row['Strike'], _T, _r, b, row['IV']), axis=1)
    data_call = pd.concat([data_call, greeks_call_df], axis=1)
    data_put = pd.concat([data_put, greeks_put_df], axis=1)

    return data_call, data_put

In [22]:
is_european = True
is_stock = True

call_MonteCarlo, put_MonteCarlo = process_data('MC', is_european, is_stock)
call_Black_Scholes_Merton, put_Black_Scholes_Merton = process_data('BS', is_european, is_stock)
callTrinomialTree, putTrinomialTree = process_data('TT', is_european, is_stock)

/var/folders/b8/6_72cd2s2vv25dcbb82ltc9m0000gn/T/ipykernel_3254/3507043702.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.tree[i, j] = self.S0 * (self.u**(j - i))
/var/folders/b8/6_72cd2s2vv25dcbb82ltc9m0000gn/T/ipykernel_3254/3507043702.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_payoffs[j] = (self.pd * payoffs[j] +
/var/folders/b8/6_72cd2s2vv25dcbb82ltc9m0000gn/T/ipykernel_3254/3507043702.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


In [23]:
call_MonteCarlo

,Strike,Call: bid,Call: offer,Call: mid,IV,Value,Delta,Gamma,Vega,Volga,Vanna,Theta
0,90.5000,1.9870,2.3990,2.1930,0.1071,2.1877,0.5977,0.0913,15.2711,-0.8801,-0.7207,-7.8160
1,91.0000,1.9500,2.2110,2.0805,0.1175,2.0800,0.5475,0.0853,15.7930,5.5663,-0.1617,-8.1796
2,91.5000,1.7440,2.0000,1.8720,0.1203,1.8705,0.5055,0.0851,15.9521,-2.8530,0.2256,-7.9947
3,92.0000,1.6160,1.7500,1.6830,0.1227,1.6829,0.4682,0.0807,15.6018,32.1530,0.4445,-8.0909
4,92.5000,1.0990,1.7530,1.4260,0.1196,1.4260,0.4260,0.0836,15.6383,6.3806,0.8110,-7.4627
5,93.0000,1.1020,1.5480,1.3250,0.1257,1.3231,0.3902,0.0779,15.4585,19.3029,1.0176,-7.5860
6,93.5000,1.0500,1.5510,1.3005,0.1356,1.3015,0.3708,0.0703,14.9920,11.4577,0.8943,-7.4608
7,94.0000,1.0020,1.3480,1.1750,0.1382,1.1752,0.3376,0.0659,14.6061,7.3357,1.4956,-6.8881
8,94.5000,0.9500,1.3510,1.1505,0.1471,1.1523,0.3216,0.0616,14.3437,20.2341,1.1715,-7.2108
9,95.0000,0.9400,1.0490,0.9945,0.1456,0.9991,0.2877,0.0586,13.5819,45.3125,1.4970,-6.9450


In [24]:
call_Black_Scholes_Merton

,Strike,Call: bid,Call: offer,Call: mid,IV,Value,Delta,Gamma,Vega,Volga,Vanna,Theta
0,90.5000,1.9870,2.3990,2.1930,0.1072,2.1930,0.5975,0.0914,15.2858,7.0333,-0.7214,-7.9955
1,91.0000,1.9500,2.2110,2.0805,0.1176,2.0805,0.5487,0.0853,15.6410,1.1556,-0.2389,-8.2081
2,91.5000,1.7440,2.0000,1.8720,0.1202,1.8720,0.5067,0.0840,15.7564,-0.0789,0.1189,-8.0939
3,92.0000,1.6160,1.7500,1.6830,0.1228,1.6830,0.4666,0.0820,15.7034,1.4744,0.4454,-7.9357
4,92.5000,1.0990,1.7530,1.4260,0.1197,1.4260,0.4244,0.0828,15.4748,5.9901,0.7954,-7.4840
5,93.0000,1.1020,1.5480,1.3250,0.1256,1.3250,0.3908,0.0774,15.1643,11.1274,1.0159,-7.4071
6,93.5000,1.0500,1.5510,1.3005,0.1357,1.3005,0.3659,0.0702,14.8598,15.0967,1.1152,-7.5381
7,94.0000,1.0020,1.3480,1.1750,0.1382,1.1750,0.3359,0.0668,14.4065,21.3813,1.2779,-7.2862
8,94.5000,0.9500,1.3510,1.1505,0.1470,1.1505,0.3167,0.0613,14.0649,24.7014,1.3113,-7.3631
9,95.0000,0.9400,1.0490,0.9945,0.1456,0.9945,0.2862,0.0591,13.4374,32.7259,1.4676,-6.8893


In [25]:
callTrinomialTree

,Strike,Call: bid,Call: offer,Call: mid,IV,Value,Delta,Gamma,Vega,Volga,Vanna,Theta
0,90.5000,1.9870,2.3990,2.1930,0.1071,2.1930,0.5968,0.0900,15.3044,6.5177,-0.7024,-7.9961
1,91.0000,1.9500,2.2110,2.0805,0.1175,2.0805,0.5473,0.0837,15.5421,-2.9182,-0.2994,-8.1755
2,91.5000,1.7440,2.0000,1.8720,0.1203,1.8720,0.5072,0.0846,15.8448,13.9434,0.2542,-8.1746
3,92.0000,1.6160,1.7500,1.6830,0.1227,1.6830,0.4683,0.0794,15.8132,-7.7124,0.3758,-7.9629
4,92.5000,1.0990,1.7530,1.4260,0.1196,1.4260,0.4235,0.0809,15.3982,-8.8304,0.6940,-7.4514
5,93.0000,1.1020,1.5480,1.3250,0.1257,1.3250,0.3923,0.0786,15.2162,26.7959,1.1219,-7.5149
6,93.5000,1.0500,1.5510,1.3005,0.1356,1.3005,0.3648,0.0689,14.6347,7.7440,1.0419,-7.4401
7,94.0000,1.0020,1.3480,1.1750,0.1382,1.1750,0.3392,0.0670,14.5106,26.1786,1.3718,-7.4507
8,94.5000,0.9500,1.3510,1.1505,0.1471,1.1505,0.3159,0.0612,13.9957,33.2525,1.3198,-7.1878
9,95.0000,0.9400,1.0490,0.9945,0.1456,0.9945,0.2883,0.0585,13.5339,26.4586,1.5660,-7.0378
